|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[50] Recommender systems with MLP projections</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300
})

# **Part 1: Tokens and MLP projections**

In [ ]:
model = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model.eval()

n_layers = model.config.n_layer

In [ ]:
text_A = "I'll tell thee everything I can; There's little to relate. I saw an aged aged man, a-sitting on a gate."
text_B = '"Who are you, aged man?" I said. "And how is it you live?"'
text_C = 'And his answer trickled through my head, like water through a sieve.'

tokens_A = tokenizer.encode()
tokens_B = tokenizer
tokens_C =

n_toks_A =
n_toks_B =
n_toks_C =

print(' Text | tokens')
print('------+-------')
print(f'  A   |  {}')
print(f'  B   |  {}')
print(f'  C   |  {}')

In [ ]:
mlp_projs = {}

def outerhook(layernum):
  def hook(module,input,output):
    mlp_projs[f'{whichtext}_{layernum}'] = output.detach().numpy()
  return hook

handles = []
for layi in range(n_layers):
  h = model.transformer.h[layi].mlp.
  handles.append(h)

In [ ]:
with torch.no_grad():
  whichtext =
  model(tokens_

  whichtext = 'B'
  model

  whichtext =

In [ ]:
for k,v in mlp_projs.items():
  print(f"mlp['{k}'] has size {list(v.shape)}")

In [ ]:
# initialize layers-by-tokens matrices
norm_A = np.zeros((,))
norm_B = np.zeros((,))
norm_C = np.zeros((,))

# calculate the norms for all tokens in all layers
for layeri in range(n_layers):
  norm_A[layeri,:] = np.linalg.norm(,axis=)
  norm_B[layeri,:] = np.linalg.norm(
  norm_C[layeri,:] = np.linalg.norm(


fig,axs = plt.subplots(1,3,figsize=(12,3.3))

# text A
cmin,cmax = np.percentile(,[,])
h = axs[0].imshow(norm_A,aspect='auto',cmap='magma',origin='lower')
axs[0].set(xlabel='Token position',ylabel='Transformer layer',title='Norms of sentence A')
fig.colorbar(h,ax=axs[0],pad=.01)

# text B
cmin,cmax =
h = axs[1].imshow()
axs[1].set(xlabel='Token position',ylabel='Transformer layer',title='Norms of sentence B')
fig.colorbar(h,ax=axs[1],pad=.01)

# text C
cmin,cmax = np.percentile(norm_C,[5,95])
h = axs[2].
axs[2].set(xlabel='Token position',ylabel='Transformer layer',title='Norms of sentence C')
fig.colorbar(h,ax=axs[2],pad=.01)


plt.tight_layout()
plt.savefig('ch7_proj50_part1.png')
plt.show()

# **Part 2: Token-average target vector recommender**

In [ ]:
whichlayer = 6

# average the vectors within each text
mean_A = np.mean(
mean_B = np.mean(

# cosine similarity between the mean vectors
cs_AB = cosine_similarity(,)

# the target vector
target_vect =

In [ ]:
_,axs = plt.subplots(1,2,figsize=(10,3.5))

axs[0].plot(,label='Vector A')
axs[0].plot(,label='Vector B')
axs[1].plot(,,'kh',markerfacecolor=[.7,.9,.9,.5])

axs[0].set(xlabel='Embeddings dimension',ylabel='Value',title='A) Mean vectors')
axs[0].legend()

axs[1].set(xlabel='Mean vector A',ylabel='Mean vector B',title=f'B) Scatter plot ($S_c$ = {cs_AB.item():.2f})')

plt.tight_layout()
plt.savefig('ch7_proj50_part2a.png')
plt.show()

In [ ]:
# cosine similarity between each token in C and the target
cossims = cosine_similarity(,)

# visualize and choose
plt.figure(figsize=(12,3))

plt.plot(cossims,'kh',markerfacecolor=[.9,.7,.9],markersize=14)
plt.axhline(,linestyle='--',linewidth=.5,color='k',zorder=-1)
plt.axvline(,linestyle='--',linewidth=.5,color='k',zorder=-1)

plt.gca().set(xticks=range(n_toks_C),xticklabels=[tokenizer.decode(i) for i in tokens_C[0,1:]],
              ylabel='Similarity with target vector')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('ch7_proj50_part2b.png')
plt.show()

# **Part 3: Recommendations through the layers**

In [ ]:
plt.figure(figsize=(10,3))
xlabels = []


for layeri in range(n_layers):

  mean_A = np.mean(
  mean_B = np.mean(

  target_vect =
  cossims = cosine_similarity

  plt.plot(layeri,,'kh',markersize=14,markerfacecolor=[.9,.7,.7])
  plt.text(,,,ha='center',fontweight='bold')


plt.gca().set(xlabel='Transformer layer',ylabel='Cosine similarity',title='Recommended token')

plt.tight_layout()
plt.savefig('ch7_proj50_part3.png')
plt.show()

# **Part 4: Recommending specific pairs**

In [ ]:
whichlayer = 6

x_A = mlp_projs[f'A_][,,]
x_B =
x_C =

cMat = np.zeros((,,))

for i in range():
  for j in range():

    # create the directional extrapolation (aka analogy) vector
    targetvect =

    # calculate cosine similarity with each C vector
    for k in range(n_toks_C):

      cMat[i,j,k] = cosine_similarity(


print(f'Matrix cMat has size {cMat.shape}, which is {np.prod(cMat.shape):,} elements!')

In [ ]:
token_idxs = np.linspace(0,n_toks_C-1,4,dtype=int)

fig,axs = plt.subplots(1,len(token_idxs),figsize=(14,5))

for tidx in range(len(token_idxs)):

  # find the best match
  idx = np.argmax(
  i,j = np.unravel_index(idx,cMat[:,:,token_idxs[tidx]].shape)

  bestpair = f'("{}"  "{}")'

  # subplot title
  title = f'Similarity with "{tokenizer.decode(tokens_C[,])}"\n{bestpair}'


  # show the full matrix
  axs[tidx].imshow(cMat[,vmin=.05,vmax=.3,aspect='auto',cmap='magma')

  # adjust the axis and image features
  axs[tidx].set(title=title,
                xticks=range(n_toks_B),xticklabels=,
                yticks=range(n_toks_A),yticklabels=)
  axs[tidx].tick_params(axis='x',labelrotation=90)


axs[0].set(xlabel='Tokens from Text B',ylabel='Tokens from Text A')

plt.tight_layout()
plt.savefig('ch7_proj50_part4.png')
plt.show()

# **Part 5: Layer-specific recommendations**

In [ ]:
max_similarities = np.zeros((,))

for layeri in range(n_layers):

  print(f'\nLayer {layeri}:')

  x_A = mlp_projs
  x_B = mlp_projs
  x_C = mlp_projs

  cMat = np.zeros((n_toks_A,n_toks_B,n_toks_C))

  for i in range(n_toks_A):
    for j in range(n_toks_B):

      # create the target vector
      targetvect =

      # calculate cosine similarity with each C vector
      for k in range(n_toks_C):
        cMat[i,j,k] =


  # populated, now find max for each C token
  for k in range(n_toks_C):
    idx = np.argmax(
    i,j = np.unravel_index(idx,cMat[:,:,k].shape)
    max_similarities[layeri,k] = np.max(

    bestpair = f'({tokenizer.decode(tokens_A[0,i+1])},{tokenizer.decode(tokens_B[0,j+1])})'
    print(f'  "{}" is paired with {}')

In [ ]:
plt.figure(figsize=(10,4))

# color limits using data percentiles
cmin,cmax =

# show the heatmap
plt.imshow()

# and make it look nicer
plt.gca().set(ylabel='Transformer layer',title='Max similarities to target vector',
              xticks=range(n_toks_C),xticklabels=[tokenizer.decode(i) for i in tokens_C[0,1:]])
plt.colorbar(pad=.01)
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('ch7_proj50_part5.png')
plt.show()